In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from plotly.subplots import make_subplots
from typing import Tuple, Union, Literal, List, Dict, Optional, Any
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor as RF
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import plotly.graph_objects as go
from utils import plot_time_series
import warnings
%load_ext autoreload
%autoreload 2
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)

# Table of Contents
1. [Preparing baseline data](#Preparing-baseline-data)
2. [Linear Model Baseline](#Linear-Model-Baseline)
3. [Random Forest Baseline](#Random-Forest-Baseline)
4. [Data Cleaning](#Data-Cleaning)
5. [Analysis of RF predictions](#Analysis-of-RF-predictions)
6. [Feature Engineering: Lagged features including Target](#Feature-Engineering:-Lagged-features-including-Target)
7. [Feature Engineering: Lagged WITHOUT including Target](#Feature-Engineering:-Lagged-features-WITHOUT-Target)
8. [Statistical rolling features](#Statistical-rolling-features)
9. [Treating Regression Error as Anomaly](#Treating-Regression-Error-as-Anomaly)
10. [Conclusions](#Conclusions)

# Preparing baseline data

In [ ]:
df = pd.read_parquet('../data/01_raw/df_train_test.parquet')
df.index = pd.to_datetime(df['Timestamps'])
df.drop(columns=['Timestamps'], inplace=True)
SEED = 42
df.columns

In [ ]:
plot_time_series(df, ['Power'], step=2)

It's a good practice to start modeling ASAP and check what kind of performance we can expect if we use pure raw data and simple models without any tuning.

It also helps us to understand the level of data cleaning and feature engineering we likely need to do.

**Drop the downtime**

There is no point using downtime in the dataset, so we just skip it.

In [ ]:
df = df[df['Power'] > 20].copy()

In [ ]:
df.head(3)

In [ ]:
df.shape

What we will do is that we split the dataset into the train and test part.

The train part will be considered as "NORMAL".

This means that we assume that there are no anomalies in that region.

Then the idea is that if the deviation of the measured Power values from the predictions are high, then this indicates that this region is likely to be an anomaly because it's drifting from the "NORMAL" training data.

We will take the first 30k points as training data. This value can be selected different but it seems to be a good compromise between having enough data from training and being far enough from the downtime where we observed the anomaly happens (which makes sense).

In [ ]:
def get_datasets(df: pd.DataFrame, target: str, split_index=30_000) -> Tuple:
    """
    Prepares train and test datasets
    """
    df_train = df[:split_index].copy()
    df_test = df[split_index:].copy()
    
    # Splitting x and y
    x_train, y_train = df_train.drop(columns=[target]), df_train[target]
    x_test, y_test = df_test.drop(columns=[target]), df_test[target]
    
    return (x_train, y_train, x_test, y_test)

In [ ]:
(x_train, 
 y_train, 
 x_test, 
 y_test
) = get_datasets(df, 'Power', 30_000)

In [ ]:
plt.figure(figsize=(20, 5))
plt.plot(y_train, label='Train set')
plt.plot(y_test, label='Test set')
plt.legend(fontsize=16)
plt.title('Train Test Split', fontsize=20)
plt.ylabel('Power', fontsize=16)

**Metrics**

For time series, it's a golden rule that for the validation/test sets we take values from the future, so we never should split data randomly.

During training and validation, we split the training set to 3 validation folds using Sliding Time Series Cross Validation. Sowe always training on the past data and validation on the future data.

We then compute both cross validation errors and the test set errors.

For the metrics, we will use MAE, RMSE and MAPE error to have different views and decide which main metric wil will use later one.

In [ ]:
def compute_metrics(y_true: np.ndarray, y_pred: np.ndarray) -> Tuple[float, float, float]:
    """
    Compute MAE, RMSE, and MAPE for regression predictions.

    Parameters
    ----------
    y_true : array-like
        Ground truth values.
    y_pred : array-like
        Predicted values.

    Returns
    -------
    tuple of floats
        (mae, rmse, mape)
    """
    y_true = np.asarray(y_true).ravel()
    y_pred = np.asarray(y_pred).ravel()

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = np.mean(np.abs((y_true - y_pred) / np.clip(np.abs(y_true), 1e-8, None))) * 100
    return mae, rmse, mape

In [ ]:
def eval_model(
    x_train: pd.DataFrame,
    y_train: pd.Series,
    x_test: pd.DataFrame,
    y_test: pd.Series,
    n_splits: int = 3,
    model_name: Literal["RF", "LinReg"] = "RF",
    model_params: Optional[Dict[str, Any]] = None,
) -> Dict[str, Any]:
    """
    Evaluate a time series model using TimeSeriesSplit cross-validation
    on the training set, then refit on the full train data and evaluate on test.
    """
    np.random.seed(SEED)

    if model_params is None:
        model_params = {}

    tscv = TimeSeriesSplit(n_splits=n_splits)
    cv_mae_list: list[float] = []
    cv_rmse_list: list[float] = []
    cv_mape_list: list[float] = []
    
    # --- Cross-validation ---
    for fold, (train_idx, val_idx) in enumerate(tscv.split(x_train), 1):
        x_train_cv = x_train.iloc[train_idx, :].copy()
        x_val_cv = x_train.iloc[val_idx, :].copy()
        y_train_cv = y_train.iloc[train_idx].copy().values.ravel()
        y_val_cv = y_train.iloc[val_idx].copy().values.ravel()

        x_scaler = StandardScaler()
        x_scaled_cv_train = x_scaler.fit_transform(x_train_cv)
        x_scaled_cv_val = x_scaler.transform(x_val_cv)

        if model_name == "RF":
            model = RF(**model_params)
        elif model_name == "LinReg":
            model = Ridge(**model_params)
        else:
            raise ValueError(f"Unknown model_name: {model_name}")

        model.fit(x_scaled_cv_train, y_train_cv)
        y_pred_cv = model.predict(x_scaled_cv_val)

        mae_err, rmse_err, mape_err = compute_metrics(y_val_cv, y_pred_cv)
        cv_mae_list.append(float(mae_err))
        cv_rmse_list.append(float(rmse_err))
        cv_mape_list.append(float(mape_err))

    cv_mae = float(np.mean(cv_mae_list))
    cv_rmse = float(np.mean(cv_rmse_list))
    cv_mape = float(np.mean(cv_mape_list))

    # --- Final model training ---
    if model_name == "RF":
        model = RF(**model_params)
    elif model_name == "LinReg":
        model = Ridge(**model_params)
    else:
        raise ValueError(f"Unknown model_name: {model_name}")

    x_scaler = StandardScaler()
    x_scaled_train = x_scaler.fit_transform(x_train)
    x_scaled_test = x_scaler.transform(x_test)

    model.fit(x_scaled_train, y_train.values.ravel())

    y_pred_test = model.predict(x_scaled_test)
    mae_err_test, rmse_err_test, mape_err_test = compute_metrics(y_test, y_pred_test)

    return {
        "cv_mae": round(cv_mae, 2),
        "cv_rmse": round(cv_rmse, 2),
        "cv_mape": round(cv_mape, 2),
        "test_mae": round(float(mae_err_test), 2),
        "test_rmse": round(float(rmse_err_test), 2),
        "test_mape": round(float(mape_err_test), 2),
        "y_pred_test": y_pred_test,
        "model": model,
    }

# Linear Model Baseline

First, we start with the simpliest model possible - Linear Model. 

If this model well, it would be a great baseline and maybe even a production model.  Let's see!

In [ ]:
# Get data
(x_train, 
 y_train, 
 x_test, 
 y_test
) = get_datasets(df, 'Power', 30_000)

# Set params
params = {
    'alpha': 0
}

eval_results  = eval_model(
    x_train,
    y_train, 
    x_test, 
    y_test, 
    3, 
    'LinReg', 
    params
)
eval_results

In [ ]:
plt.figure(figsize=(20, 5))
plt.plot(y_test.values[:1000], label='True')
plt.plot(eval_results['y_pred_test'][:1000], label='Prediction')
plt.legend()

- In general, the predictions do not look good. It seems be overfitted and having high variance.
- Also, as we already have seen in EDA, our taregt has outliers, so it can be a big problem, especially for a linear model.
- ALWAYS, to make better conclusions, zoom in. Let's do with Plotly.

In [ ]:
def plot_predictions(y_true: pd.Series, y_pred: pd.Series) -> None:
    """
    Plots interactive predictions with Plotly
    """
    # Assume y_test and y_pred are pandas Series with a datetime index
    fig = go.Figure()
    
    # Add actual values
    fig.add_trace(go.Scatter(
        x=y_true.index, y=y_true.values,
        mode='lines',
        name='Actual',
        line=dict(width=2)
    ))
    
    # Add predicted values
    fig.add_trace(go.Scatter(
        x=y_true.index, y=y_pred,
        mode='lines',
        name='Predicted',
        line=dict(width=1)
    ))
    
    # Customize layout
    fig.update_layout(
        title='Actual vs Predicted Values',
        xaxis_title='Date',
        yaxis_title='Value',
        legend=dict(x=0, y=1),
        height=500
    )
    
    fig.show()

In [ ]:
plot_predictions(y_test, eval_results['y_pred_test'])

Let's try to add some regularization.

In [ ]:
# Get data
(x_train, 
 y_train, 
 x_test, 
 y_test
) = get_datasets(df, 'Power', 30_000)

# Set params
params = {
    'alpha': 10
}

eval_results  = eval_model(
    x_train,
    y_train, 
    x_test, 
    y_test, 
    3, 
    'LinReg', 
    params
)
eval_results

We see that adding regularization does not help much.

It seems that with such noisy but dense data, it will be hard to fit a linear model, at least, without an extensive cleaning.

It can also be the case that our problem is non-linear.

This actually makes sense because we saw visually and even during PCA, the first 2 components did not explain even 80% of the variance which also indicates the problem non-linearity.

Let's fit a non-linear model, e.g. Random Forest.

# Random Forest Baseline

In [ ]:
# Get data
(x_train, 
 y_train, 
 x_test, 
 y_test
) = get_datasets(df, 'Power', 30_000)

params = {
    'n_estimators': 100, 
    'random_state': SEED,
    'n_jobs':-1
}

eval_results  = eval_model(
    x_train,
    y_train, 
    x_test, 
    y_test, 
    3, 
    'RF', 
    params
)
eval_results

We see a sudden improvement. That's great! Let's see what the predictions look like.

In [ ]:
plot_predictions(y_test, eval_results['y_pred_test'])

We still see high variance but the predictions are way more stable now.

Also, we can easily see that the predictions do not go below the bottom level out of the box but the linear model did not manage to do that.

Let's also check how the built-in feature importance looks like.

In [ ]:
# Extract values
importances = eval_results['model'].feature_importances_
cols = x_train.columns

# Combine into DataFrame
feat_imp = (
    pd.DataFrame({
        "feature": cols,
        "importance": importances
    })
    .sort_values("importance", ascending=False)
    .reset_index(drop=True)
)

feat_imp

Among the TOP 2 features, we see the same ones that we observed in our EDA analysis.

We also see that many features are WAY less important. Let's keep it in mind for further modelling.

Among the two baselines, it's obvious we need to go with Random Forest and explore it further because:
- It has smaller error
- It doesn't produce values below the low limit like Linear Regression.

We also see that the spikes are not predicted which is a good sign - the model does not get overfitted (it could be different in case of Gradient Boosting or Neural Network, for example).

However, in EDA we saw that features and the target itself have outliers which can negatively impact the model accuracy.

Let's see if basic outlier removal would help.

# Data Cleaning

First, it's crucial to deal with outliers in target.

In general, by properly cleaning the target, we can achieve one of the highest boosts in model performance.

This is especially the case in this problem, because measurements are very noisy and have many outliers.

## Removing Outliers in target

For quick iterations between different experiments, it's better to read the data again in each section.

This reduces a change to operate with the dataframe that has already been modified.

In [ ]:
df = pd.read_parquet('../data/01_raw/df_train_test.parquet')
df.index = pd.to_datetime(df['Timestamps'])
df.drop(columns=['Timestamps'], inplace=True)
df = df[df['Power'] > 20].copy()

In [ ]:
(x_train, 
 y_train,  
 x_test, 
 y_test
) = get_datasets(df, 'Power')

In [ ]:
sns.histplot(y_train)
plt.ylim(0, 20)

We clearly see that there are outliers. Let's remove them using z-score filtering.

For now, we

In [ ]:
# Z-score visualization
mean = df['Power'].mean()
std = df['Power'].std(ddof=0)
z_score = np.abs((df['Power'] - mean) / std)
sns.histplot(z_score)
plt.xlim(0, 5)

In [ ]:
def remove_outliers_zscore(
    df: pd.DataFrame,
    threshold: float = 3.0,
    nan_treatment: str = 'ffill',
    stats: Optional[Dict[str, Tuple[float, float]]] = None
) -> Tuple[pd.DataFrame, Dict[str, Tuple[float, float]]]:
    """
    Replace outliers (per-column z-score > threshold) with NaN.
    Optionally forward-fills or drops the resulting NaNs.
    If stats are provided, uses them; otherwise computes and returns them
    (so the same parameters can be applied to test data).

    Parameters
    ----------
    df : pd.DataFrame
        Input data (numeric/mixed).
    threshold : float
        |z| cutoff.
    nan_treatment : {'ffill','drop'}
        How to handle NaNs.
    stats : dict or None
        Precomputed {col: (mean, std)}.

    Returns
    -------
    df_masked : pd.DataFrame
        DataFrame with outliers replaced and NaN treatment applied.
    fit_stats : Dict[str,(float,float)]
        Mean and std used (save for test).
    """
    df_masked = df.copy()
    numeric_cols = df.select_dtypes(include=np.number).columns
    fit_stats: Dict[str, Tuple[float, float]] = {}

    if stats is None:
        for col in numeric_cols:
            fit_stats[col] = (df[col].mean(), df[col].std(ddof=0))
    else:
        fit_stats = stats

    total_changed = 0

    for col in numeric_cols:
        mean, std = fit_stats[col]
        if std == 0:
            print(f"{col}: std==0, skipped")
            continue

        z = np.abs((df[col] - mean) / std)
        changed = int((z > threshold).sum())
        total_changed += changed

        print(f"{col}: {changed} replaced")

        df_masked.loc[z > threshold, col] = np.nan

    print(f"TOTAL replaced: {total_changed}")

    if nan_treatment == 'ffill':
        df_masked = df_masked.ffill()
    elif nan_treatment == 'drop':
        df_masked = df_masked.dropna()
    else:
        raise ValueError(f"nan_treatment '{nan_treatment}' not recognized.")

    return df_masked, fit_stats

In [ ]:
y_train_clean, _ = remove_outliers_zscore(pd.DataFrame(y_train), nan_treatment='ffill', threshold=3)

In [ ]:
117 / y_train.shape[0]*100

So, we replaced only 117 values (or 0.39%).

Let's see how this influences the model performance>

In [ ]:
sns.histplot(y_train, label='Raw Data')
sns.histplot(y_train_clean.values.ravel(), label='Cleaned Data')
plt.legend()
# plt.ylim(0, 50)

In [ ]:
params = {
    'n_estimators': 100, 
    'random_state': SEED,
    'n_jobs':-1
}

eval_results  = eval_model(
    x_train,
    y_train_clean, 
    x_test, 
    y_test, 
    3, 
    'RF', 
    params
)
eval_results

In [ ]:
# 'cv_mae': 70.0,
#  'cv_rmse': 113.31,
#  'cv_mape': 8.15,
#  'test_mae': 74.51,
#  'test_rmse': 122.62,
#  'test_mape': 9.76,

We see that we reduced the Cross-Validation errors a lot which makes sense because we don't compute the errors for the outliers now.

But what is also important to notice is that we reduced the test errors qute a lot EVEN THOUGH we haven't removed the outliers there.

So, removing outliers that cover 0.5% of the data is crucial and can produce better results that endless model tuning.

Now, let's check outliers in features.

## Outliers removal in features

In [ ]:
df = pd.read_parquet('../data/01_raw/df_train_test.parquet')
df.index = pd.to_datetime(df['Timestamps'])
df.drop(columns=['Timestamps'], inplace=True)
df = df[df['Power'] > 20].copy()

In [ ]:
(x_train, 
 y_train,  
 x_test, 
 y_test
) = get_datasets(df, 'Power')

In [ ]:
# Select numeric columns only
numeric_cols = x_train.select_dtypes(include=np.number).columns
n_cols = 3
n_rows = int(np.ceil(len(numeric_cols) / n_cols))

plt.figure(figsize=(15, 10))

for i, col in enumerate(numeric_cols, 1):
    plt.subplot(n_rows, n_cols, i)
    sns.histplot(x_train[col].dropna(), bins=20, kde=True)
    plt.title(col, fontsize=10)
    plt.xlabel('')
    plt.ylabel('')

plt.tight_layout()

For simplicity and comparison purposes, we will clean the outliers only in the train set.

However, if the whole idea of removing outliers work, we would need to clean these outliers in the test (aka production) data too.

This is because when the model deployed, we want to have the same data preprocessing that we use to train the model.

In [ ]:
y_train_clean, _ = remove_outliers_zscore(pd.DataFrame(y_train), nan_treatment='ffill', threshold=3)

In [ ]:
x_train_clean, z_score_stats = remove_outliers_zscore(x_train, nan_treatment='ffill', threshold=3)

Since we are cleaning features, it's essential to use the same cleaning on both training and test sets.

In this particular filter, we save the mean and std statistics from the training set and then apply them on the test set.

**If we do not do that, we would introduce data leackage because we would use the infromation from the future on the test set**

In [ ]:
# Demonstrate drop as well
x_test_clean, _ = remove_outliers_zscore(x_test, nan_treatment='ffill', threshold=3, stats=z_score_stats)

In [ ]:
# Select numeric columns only
numeric_cols = x_train_clean.select_dtypes(include=np.number).columns
n_cols = 3
n_rows = int(np.ceil(len(numeric_cols) / n_cols))

plt.figure(figsize=(15, 10))

for i, col in enumerate(numeric_cols, 1):
    plt.subplot(n_rows, n_cols, i)
    sns.histplot(x_train_clean[col].dropna(), bins=20, kde=True)
    plt.title(col, fontsize=10)
    plt.xlabel('')
    plt.ylabel('')

plt.tight_layout()

In [ ]:
params = {
    'n_estimators': 100, 
    'random_state': SEED,
    'n_jobs':-1
}

eval_results  = eval_model(
    x_train_clean,
    y_train_clean, 
    x_test_clean, 
    y_test, 
    3, 
    'RF', 
    params
)
eval_results

In [ ]:
# 'cv_mae': 65.02,
#  'cv_rmse': 95.33,
#  'cv_mape': 7.64,
#  'test_mae': 71.88,
#  'test_rmse': 120.91,
#  'test_mape': 9.29,

We see that the error is not influenced much in this case.

In fact, it became slightly worse.

This is true for this particular dataset.

However, this might NOT be true for many-many other datasets.

So, removing outliers in features IN ADDITION to the target is a MUST step to check in any time series-based analysis.

In this case, we WILL implement outlier removal in features as well, so you know how to implement this in production and then later in your work projects.

In [ ]:
plot_predictions(y_test, eval_results['y_pred_test'])

### Outlier dropping vs ffilling

Pure data dropping is usually not good because it's harder to maintain in production (making sure that the indicies are aligned, etc) and we lose information.

Unless we get a very good improvement, it's better to stick with some filling or interpolation options.

On the other hand, forward filling might not be an ideal option because it create data that in realily never existed.

You can fill with other options but:

1. It's gives headache in production
2. Again, you will create data that never existed.

For now, let's just make forward fill because then we don't need to make sure that X and y indicies match (twice!).

However, it's a good execise to test dropping the indicies for the outliying values and see how the model behaves, so feel free!

# Analysis of RF predictions

Ok, now as we have built the first robust baseline, we can start analyzing the predictions in more detail.

Let's prepare the function that plots the actual values VS ML model predictions.

Also, since our dataset is very noisy and we don't clean (for now) the test set, the erro value can be noisy as well.

So, we introduce the median rolling to see the trends better.

In [ ]:
def plot_errors(
    x_true: pd.DataFrame,
    y_true: Union[pd.Series, pd.DataFrame],
    y_pred: Union[np.ndarray, list],
    error: Literal["mae", "mape"],
    error_threshold: float,
    rolling_window: int
) -> None:
    """
    Plot prediction errors with detected anomalies and threshold lines.

    Parameters
    ----------
    x_true : pd.DataFrame
        Original input data (with datetime index).
    y_true : pd.Series or pd.DataFrame
        True target values.
    y_pred : array-like
        Predicted target values.
    error : {"mae", "mape"}
        Type of error to visualize.
    error_threshold : float
        Threshold above which points are flagged as anomalies.
    rolling_window: int
        Rollin window for the error rolling aggregation
    """

    # Create a DataFrame with true values and compute the error
    y_test_err = pd.DataFrame(y_true)
    if error == "mae":
        y_test_err['Error'] = abs(y_test_err['Power'] - y_pred)
    elif error == 'rmse':
        y_test_err['Error'] = np.sqrt((y_test_err['Power'] - y_pred)**2)
    elif error == 'mape':
        y_test_err['Error'] = abs(y_test_err['Power'] - y_pred) / y_test_err['Power'] * 100

    # Ensure both y_test_err and x_true have datetime indices
    y_test_err.index = pd.to_datetime(y_test_err.index)
    x_true.index = pd.to_datetime(x_true.index)

    # Initialize Plotly figure
    fig = go.Figure()

    # Plot the error over time
    fig.add_trace(go.Scatter(
        x=y_test_err.index, 
        y=y_test_err['Error'].values,
        mode='lines',
        name='Error',
        line=dict(width=2)
    ))

    # Add horizontal line for static 95th percentile threshold
    fig.add_hline(
        y=error_threshold,
        line_color="black",
        annotation_text="Upper Threshold",
        annotation_position="bottom right"
    )

    # Plot rolling median error for smoothed trend
    fig.add_trace(go.Scatter(
        x=y_test_err.index,
        y=y_test_err['Error'].rolling(rolling_window).median(),
        mode='lines',
        name='Rolling Median Error',
        line=dict(width=2)
    ))

    # Configure layout settings
    fig.update_layout(
        title='Prediction Error and Anomaly Detection',
        xaxis_title='Date',
        yaxis_title='Error',
        legend=dict(x=0, y=1),
        height=500,
        template='plotly_white'
    )

    fig.show()

In [ ]:
plot_errors(x_test, y_test, eval_results['y_pred_test'], error='mae', error_threshold=100, rolling_window=228) # 72, 144

In [ ]:
plot_errors(x_test, y_test, eval_results['y_pred_test'], error='mape', error_threshold=10, rolling_window=228) # 72, 144

In [ ]:
mae_error = abs(y_test - eval_results['y_pred_test'])
mae_error_rol = mae_error.rolling(144).median()

In [ ]:
mape_error = abs(y_test - eval_results['y_pred_test']) / y_test * 100
mape_error_rol = mape_error.rolling(144).median()

In [ ]:
sns.histplot(mape_error, label='Raw MAPE')
sns.histplot(mape_error_rol, label='Rolling MAPE')
plt.legend()
plt.xlim(0, 30)

We see that the rolling value is much more stable and it;s also easier to select t=a threshold above which we can consider the errors to show anomalies. 

In [ ]:
np.quantile(mape_error_rol.dropna(), 0.93)

Generally, we can have 2 options:
1. Fix the threshold value.
2. Compute it as a quantile.

For simplicity, we can keep just the value and then switch if we want to.

**Preliminary Conclusions**

We can see that we can identify the anomaly region about 18-19 days in advance which is quite good.

The number of days to value is out business metric and THIS is the metric we need to optimize.

For experiments, we can still for now leave Cross Validation ML metrics with the assumption that the better metrics, the mode stable and accurate predictions of out business metrics are.

Let's iterate using the ML metrics and for some promising models compute the Time-to-Failure metric.

# Feature Engineering: Lagged features including Target

Before computing features, it's highly recommended that we remove the outliers BEFORE it.

Otherwise, we would need to remove outliers twice which can introduce even more noise to the data.

In [ ]:
# Read again for reproducibility
df = pd.read_parquet('../data/01_raw/df_train_test.parquet')
df = df[df['Power'] > 20].copy()
df.index = pd.to_datetime(df['Timestamps'])
df.drop(columns=['Timestamps'], inplace=True)
# First, we need to get the training dataset
df_train = df[:30_000]
df_test = df[30_000:]

In [ ]:
df_clean_train, z_score_stats = remove_outliers_zscore(df_train, threshold=3, nan_treatment='ffill')

In [ ]:
# Cleaning the features in the test set
features = [col for col in df.columns if col not in ['Power']]
x_clean_test, _ = remove_outliers_zscore(df_test[features], threshold=3, nan_treatment='ffill', stats=z_score_stats)

In [ ]:
df_clean_test = pd.concat([x_clean_test, df_test['Power']], axis=1)

Now let's add lag features

In [ ]:
def add_lag_features(df: pd.DataFrame, columns:List[str]=None, lags: List[int]=[1], drop_na=True):
    """
    Add lag features to DataFrame.
    
    Parameters:
    - df: DataFrame
    - columns: list of column names (default: all numeric)
    - lags: int or list of lag periods (default: 1)
    - drop_na: bool, drop NaN rows (default: True)
    
    Returns: DataFrame with lag features
    """
    df_result = df.copy()
    
    # Create lag features
    for col in columns:
        for lag in lags:
            df_result[f"{col}_lag{lag}"] = df_result[col].shift(lag)
    
    if drop_na:
        return df_result.dropna()
    else:
        return df_result.bfill()  # Backward fill NaNs

In [ ]:
# Adding lagged features
df_train_feat = add_lag_features(df_clean_train, columns=['Power', 'GenRPM'], lags=[1, 2, 3], drop_na=False)
x_test_feat = add_lag_features(df_clean_test, columns=['Power', 'GenRPM'], lags=[1, 2, 3], drop_na=False)

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(df_clean_train['Power'][:200], label='Raw Signal')
plt.plot(df_train_feat['Power_lag1'][:200], label='Lagged Signal')
plt.legend(fontsize=16)

In [ ]:
# Assume y_test and y_pred are pandas Series with a datetime index
fig = go.Figure()

# Add actual values
fig.add_trace(go.Scatter(
    x=x_train.index, y=df_train['Power'].values,
    mode='lines',
    name='Actual',
    line=dict(width=2)
))

# Add predicted values
fig.add_trace(go.Scatter(
    x=x_train.index, y=df_clean_train['Power'].values,
    mode='lines',
    name='Predicted',
    line=dict(width=2)
))

# Customize layout
fig.update_layout(
    title='Actual vs Cleaned Values',
    xaxis_title='Date',
    yaxis_title='Value',
    legend=dict(x=0, y=1),
    height=500
)

fig.show()

It seems that the outliers are cleaned quite well.

However, let's look at the cleaned data alone.

In [ ]:
plot_time_series(df_clean_train, ['Power'], step=2)

Even though the outliers are similar in nature, z-score filter cannot capture it.

This is important because we add lagged features, so if the data is not cleaned well, we create additional outlying values to the training data.

We will consider this fact when we do more in-depth cleaning.

Note how we add the features only AFTER the data split into the train and test to:

1. Avoid data leakage into the future.
2. Slowly preparing the code and flow that we will then put to the production pipelines.

In [ ]:
features = [col for col in df_train_feat.columns if col != 'Power'] # Note that we keep Power_lagged features
# Train
x_clean_train = df_train_feat[features].copy()
y_clean_train = df_train_feat['Power'].copy()

# Test
x_clean_test = x_test_feat[features].copy()
y_test = df_test['Power'].copy()

In [ ]:
params = {
    'n_estimators': 100, 
    'random_state': SEED,
    'n_jobs':-1
}

eval_results  = eval_model(
    x_clean_train,
    y_clean_train, 
    x_clean_test, 
    y_test, 
    3, 
    'RF', 
    params
)
eval_results

In [ ]:
# 'cv_mae': 65.08,
#  'cv_rmse': 95.42,
#  'cv_mape': 7.65,
#  'test_mae': 71.44,
#  'test_rmse': 120.68,
#  'test_mape': 9.22,

Nice! We see that we significanlty the errors.

Let's see now how this looks in terms of Time to Failure.

In [ ]:
plot_errors(x_test, y_test, eval_results['y_pred_test'], error='mae', error_threshold=40, rolling_window=288)

In [ ]:
plot_errors(x_test, y_test, eval_results['y_pred_test'], error='mape', error_threshold=6, rolling_window=288)

In [ ]:
plot_predictions(y_test, eval_results['y_pred_test'])

Very often adding lagged features help a lot.

In this case, it also helped to reduce the error.

However, in our case, even though the predictions are better, the problem is that we start including the "degraded" parameter into the model as the feature.

This results in the fact that it becomes HARDER to see the anomaly region and distinguish it from the normal region.

All this can make it confusing to interpret the model behavior.

Let's NOT include Power as a feature and instead include some other parameters.

**THIS SHOWS AN EXAMPLE WHEN BETTER ML METRICS DO NOT RESULT IN BETTER BUSINESS METRICS.**

# Feature Engineering: Lagged features WITHOUT Target

In [ ]:
# Read again for reproducibility
df = pd.read_parquet('../data/01_raw/df_train_test.parquet')
df = df[df['Power'] > 20].copy()
df.index = pd.to_datetime(df['Timestamps'])
df.drop(columns=['Timestamps'], inplace=True)
# First, we need to get the training dataset
df_train = df[:30_000]
df_test = df[30_000:]

In [ ]:
df_clean_train, z_score_stats = remove_outliers_zscore(df_train, threshold=3, nan_treatment='ffill')

In [ ]:
# Cleaning the features in the test set
features = [col for col in df.columns if col not in ['Power']]
x_clean_test, _ = remove_outliers_zscore(df_test[features], threshold=3, nan_treatment='ffill', stats=z_score_stats)

In [ ]:
df_clean_test = pd.concat([x_clean_test, df_test['Power']], axis=1)

In [ ]:
# Adding lagged features
df_train_feat = add_lag_features(df_clean_train, columns=['GenRPM', 'GenPh1Temp'], lags=[1, 2, 3], drop_na=False)
x_test_feat = add_lag_features(df_clean_test, columns=['GenRPM', 'GenPh1Temp'], lags=[1, 2, 3], drop_na=False)

In [ ]:
features = [col for col in df_train_feat.columns if col != 'Power']
# Train
x_clean_train = df_train_feat[features].copy()
y_clean_train = df_train_feat['Power'].copy()

# Test
x_clean_test = x_test_feat[features].copy()
y_test = df_test['Power'].copy()

In [ ]:
params = {
    'n_estimators': 100, 
    'random_state': SEED,
    'n_jobs':-1
}

eval_results  = eval_model(
    x_clean_train,
    y_clean_train, 
    x_clean_test, 
    y_test, 
    3, 
    'RF', 
    params
)
eval_results

In [ ]:
# 'cv_mae': 65.02,
#  'cv_rmse': 95.33,
#  'cv_mape': 7.64,
#  'test_mae': 71.88,
#  'test_rmse': 120.91,
#  'test_mape': 9.29,

Cool! We see that we still reduced the error but hopefully we have NOT deteriorated the anomaly class close to the downtime.

In [ ]:
plot_errors(x_test, y_test, eval_results['y_pred_test'], error='mape', error_threshold=8, rolling_window=228)

We see tht we can still clearly distiguish the anomaly class AND we reduced the cross-validation error.

It seems that this is a perfect mix.

# Statistical rolling features

Now, let's add basic statistical features

In [ ]:
def add_rolling_features(df, columns, window_sizes=7, stats=['mean', 'median'], drop_na=True):
    """
    Add rolling features to DataFrame.
    
    Parameters:
    - df: DataFrame
    - columns: list of column names
    - window_sizes: int or list of window sizes (default: 7)
    - stats: list of statistics ['mean', 'median', 'std', 'min', 'max', 'skew', 'kurt']
    - drop_na: bool, drop NaN rows (default: True)
    
    Returns: DataFrame with rolling features
    """
    df_result = df.copy()
    
    # Convert single values to lists
    if isinstance(window_sizes, int):
        window_sizes = [window_sizes]
    if isinstance(columns, str):
        columns = [columns]
    
    # Create rolling features
    for col in columns:
        for window in window_sizes:
            rolling = df_result[col].rolling(window)
            
            for stat in stats:
                if stat == 'mean':
                    df_result[f"{col}_roll{window}_mean"] = rolling.mean()
                elif stat == 'median':
                    df_result[f"{col}_roll{window}_median"] = rolling.median()
                elif stat == 'std':
                    df_result[f"{col}_roll{window}_std"] = rolling.std()
                elif stat == 'min':
                    df_result[f"{col}_roll{window}_min"] = rolling.min()
                elif stat == 'max':
                    df_result[f"{col}_roll{window}_max"] = rolling.max()
                elif stat == 'skew':
                    df_result[f"{col}_roll{window}_skew"] = rolling.skew()
                elif stat == 'kurt':
                    df_result[f"{col}_roll{window}_kurt"] = rolling.kurt()
    if drop_na:
        return df_result.dropna()
    else:
        return df_result.bfill()  # Backward fill NaNs

In [ ]:
# Read again for reproducibility
df = pd.read_parquet('../data/01_raw/df_train_test.parquet')
df = df[df['Power'] > 20].copy()
df.index = pd.to_datetime(df['Timestamps'])
df.drop(columns=['Timestamps'], inplace=True)
# First, we need to get the training dataset
df_train = df[:30_000]
df_test = df[30_000:]

In [ ]:
df_clean_train, z_score_stats = remove_outliers_zscore(df_train, threshold=3, nan_treatment='ffill')
features = [col for col in df.columns if col not in ['Power']]
x_clean_test, _ = remove_outliers_zscore(df_test[features], threshold=3, nan_treatment='ffill', stats=z_score_stats)
df_clean_test = pd.concat([x_clean_test, df_test['Power']], axis=1)

In [ ]:
# Adding lagged features
df_clean_train = add_lag_features(df_clean_train, columns=['GenRPM', 'GenPh1Temp'], lags=[1, 2, 3], drop_na=False)
df_clean_test = add_lag_features(df_clean_test, columns=['GenRPM', 'GenPh1Temp'], lags=[1, 2, 3], drop_na=False)

In [ ]:
df_train_feat =  add_rolling_features(
    df_clean_train,
    window_sizes=[5, 10, 15],
    columns=['GenRPM', 'WindSpeed', 'GenPh1Temp'], 
    stats=['median', 'std', 'min', 'max'],
    drop_na=False
)

In [ ]:
x_test_feat =  add_rolling_features(
    df_clean_test,
    window_sizes=[5, 10, 15],
    columns=['GenRPM', 'WindSpeed', 'GenPh1Temp'],
    stats=['median', 'std', 'min', 'max'],
    drop_na=False
)

In [ ]:
features = [col for col in df_train_feat.columns if col != 'Power']
# Train
x_clean_train = df_train_feat[features].copy()
y_clean_train = df_train_feat['Power'].copy()

# Test
x_clean_test = x_test_feat[features].copy()
y_test = df_test['Power'].copy()

In [ ]:
eval_results  = eval_model(
    x_clean_train,
    y_clean_train, 
    x_clean_test, 
    y_test, 
    3, 
    'RF', 
    params
)
eval_results

In [ ]:
# 'cv_mae': 56.32,
#  'cv_rmse': 82.79,
#  'cv_mape': 6.64,
#  'test_mae': 65.32,
#  'test_rmse': 115.11,
#  'test_mape': 8.39,

In [ ]:
plot_errors(x_test, y_test, eval_results['y_pred_test'], error='mape', error_threshold=8, rolling_window=288)

We see that overall, it seems we improved the model performance.

This can potentially confirm the fact that the noise in features make our model performance worse and when adding rolling features, we reduce this noise and improve the model performance.

# Treating Regression Error as Anomaly

In [ ]:
plot_predictions(y_test, eval_results['y_pred_test'])

In [ ]:
plot_errors(x_test, y_test, eval_results['y_pred_test'], error='mape', error_threshold=8, rolling_window=288)

In [ ]:
mape_error = abs(y_test - eval_results['y_pred_test']) / y_test * 100
mape_error_rol = mape_error.rolling(288).median()

In [ ]:
sns.histplot(mape_error_rol)
plt.xlim(0, 30)
plt.xlabel('MAPE TEST SET')

In [ ]:
np.quantile(mape_error_rol.dropna(), 0.92)

### Let's compute how much time in advance we can detect the anomaly

In [ ]:
def detect_anomaly(
    y_true: np.ndarray | pd.Series,
    y_pred: np.ndarray | pd.Series,
    threshold: float,
    waiting_time: int,
    rolling_window: int,
    error_type: Literal['raw','rolling'] = 'raw'
) -> np.ndarray:
    """
    Detect anomalies based on prediction error exceeding a threshold.
    Optionally applies a rolling-median smoothing to detect slow deviations.
    Requires a minimum number of consecutive violations (waiting_time)
    before confirming an anomaly.

    Parameters
    ----------
    y_true : array-like
        Ground-truth target values.
    y_pred : array-like
        Model predictions.
    threshold : float
        Error value above which a point is considered suspicious.
    waiting_time : int
        Number of consecutive threshold violations needed to flag anomaly.
    rolling_window : int
        Window size for rolling-median smoothing (used when error_type='rolling').
    error_type : {'raw','rolling'}, default='raw'
        - 'raw'     : use absolute error directly
        - 'rolling' : use rolling median of absolute error to detect slow drift

    Returns
    -------
    anomalies : np.ndarray of bool
        Boolean array marking anomaly positions.
    """

    # absolute error
    error = np.abs(np.asarray(y_true) - np.asarray(y_pred))/np.asarray(y_true) * 100

    if error_type == 'rolling':
        # rolling median (slow drift detection)
        error = (
            pd.Series(error)
            .rolling(rolling_window, min_periods=1)
            .median()
            .to_numpy()
        )

    above = error > threshold
    anomalies = np.zeros_like(above, dtype=bool)

    # consecutive violations
    count = 0
    for i, flag in enumerate(above):
        count = count + 1 if flag else 0
        if count >= waiting_time:
            anomalies[i] = True

    return anomalies

In [ ]:
# let's make delay 1 days: 10 mins * 6 points * 24
anomalies = detect_anomaly(
    y_true=y_test, 
    y_pred=eval_results['y_pred_test'],
    threshold=8.5,
    waiting_time=72, 
    rolling_window=288, 
    error_type='rolling'
)

In [ ]:
anomalies.sum()

In [ ]:
def plot_prediction_with_anomalies(
    x_true,
    y_true,
    y_pred,
    error,
    error_threshold,
    rolling_window,
    anomalies=None
):
    """
    Plot prediction errors with detected anomalies and threshold line.

    Parameters
    ----------
    x_true : pd.DataFrame
        Original input data (with index aligned to y_true).
    y_true : pd.Series or pd.DataFrame
        True target values (must contain 'Power' column if DataFrame).
    y_pred : array-like
        Predicted target values (same length as y_true).
    error : str
        Type of error to visualize. Either 'mae' or 'mape'.
    error_threshold : float
        Static threshold value to draw as a horizontal line.
    rolling_window: int
        Rolling windown for the error
    anomalies : array-like of bool, optional
        Boolean mask (same length as y_true) where True indicates anomaly.
    """

    # --- Build y_test_err DataFrame with 'Power' + 'Error' ---

    # Ensure y_true is DataFrame with column 'Power'
    if isinstance(y_true, pd.Series):
        y_test_err = y_true.to_frame(name='Power').copy()
    else:
        y_test_err = y_true[['Power']].copy()

    # Compute error
    if error == "mae":
        y_test_err['Error'] = (y_test_err['Power'] - y_pred).abs()
    elif error == 'mape':
        y_test_err['Error'] = (y_test_err['Power'] - y_pred).abs() / (y_test_err['Power'] + 1e-8) * 100
    else:
        raise ValueError("error must be 'mae' or 'mape'")

    # --- Handle anomalies mask robustly ---

    if anomalies is not None:
        anomalies = np.asarray(anomalies).astype(bool)

        # Align length by trimming from the front if needed
        if len(anomalies) > len(y_test_err):
            anomalies = anomalies[-len(y_test_err):]
        elif len(anomalies) < len(y_test_err):
            # pad with False at the beginning
            pad = len(y_test_err) - len(anomalies)
            anomalies = np.concatenate([np.zeros(pad, dtype=bool), anomalies])

        # Now lengths match exactly
        y_test_err['Anomaly'] = anomalies
    else:
        y_test_err['Anomaly'] = False

    print(f"Number of anomalies: {y_test_err['Anomaly'].sum()}")

    # --- Build figure ---

    fig = go.Figure()

    # 1) Error line
    fig.add_trace(go.Scatter(
        x=y_test_err.index,
        y=y_test_err['Error'].values,
        mode='lines',
        name='Error',
        line=dict(width=2)
    ))

    # 2) Threshold line
    fig.add_hline(
        y=error_threshold,
        line_color="black",
        annotation_text="Upper Threshold",
        annotation_position="bottom right"
    )

    # 3) Rolling median error
    fig.add_trace(go.Scatter(
        x=y_test_err.index,
        y=y_test_err['Error'].rolling(rolling_window, min_periods=1).median(),
        mode='lines',
        name='Rolling Median Error',
        line=dict(width=2)
    ))

    # 4) Anomaly markers
    anomaly_mask = y_test_err['Anomaly']
    if anomaly_mask.any():
        fig.add_trace(go.Scatter(
            x=y_test_err.index[anomaly_mask],
            y=y_test_err.loc[anomaly_mask, 'Error'].rolling(rolling_window, min_periods=1).median(),
            mode='markers',
            name='Anomaly',
            marker=dict(size=8, color='red', symbol='circle-open')
        ))

    # Layout
    fig.update_layout(
        title='Prediction Error and Anomaly Detection',
        xaxis_title='Index',
        yaxis_title='Error',
        legend=dict(x=0, y=1),
        height=500,
        template='plotly_white'
    )

    fig.show()

In [ ]:
threshold = 8.5
rolling_window = 288
waiting_time = 72

anomalies = detect_anomaly(y_test, eval_results['y_pred_test'], threshold, waiting_time, rolling_window, error_type='rolling')

plot_prediction_with_anomalies(
    x_true=x_test,
    y_true=y_test,
    y_pred=eval_results['y_pred_test'],
    error='mape',
    error_threshold=threshold,
    rolling_window=rolling_window,
    anomalies=anomalies
)

In [ ]:
anomaly_timestamps = y_test.iloc[anomalies].index
detected_anomaly_start = anomaly_timestamps[0]
stoppage_time = pd.Timestamp('2008-06-05 15:00:00')
stoppage_time - detected_anomaly_start

So, we see that with out baseline model, we can **ROBUSTLY** estimate the anomaly with Time-to-Failure of 19.5 days. 

# Conclusions

1. We created a good baseline model AND approach for detecting anomalies based on Random Forest Regression model.
2. We improved model performance through basic data cleaning of features and feature engineering.
3. The estimated Time-to-Failure on the test set is 19.5 days.
4. Data cleaning and basic feature engineering improves the model performance a lot comapred to the model fitted on the raw data.
5. The linear model got bad performance, so we did not proceed with it.
6. The next step is to find the best model and start registering hyperparameter tuning and model performance in MLflow for reproducibility. 